In [1]:
import requests
import json

test_prompt = "The dog trainer said Buddy is making great progress with 'stay' command"
lmstudio_url = "http://localhost:1234/v1/chat/completions"

# Define JSON Schemas for structured outputs compatible with LM Studio
CURATOR_SCHEMA = {
    "name": "curator_output",
    "schema": {
        "type": "object",
        "additionalProperties": True
    }
}

THINKER_SCHEMA = {
    "name": "thinking_response",
    "schema": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
            "internal_reasoning": {"type": "string"},
            "verified_facts": {"type": "array", "items": {"type": "string"}},
            "assumptions_identified": {"type": "array", "items": {"type": "string"}},
            "patterns_observed": {"type": "array", "items": {"type": "string"}},
            "information_gaps": {"type": "array", "items": {"type": "string"}},
            "user_emotional_state": {"type": "string"},
            "situation_summary": {"type": "string"},
            "response_guidance": {
                "type": "object",
                "additionalProperties": False,
                "properties": {
                    "tone_recommendation": {
                        "type": "string",
                        "enum": [
                            "celebratory",
                            "supportive",
                            "informational",
                            "problem-solving"
                        ]
                    },
                    "key_points_to_address": {"type": "array", "items": {"type": "string"}},
                    "questions_to_explore": {"type": "array", "items": {"type": "string"}},
                    "avoid_assumptions": {"type": "array", "items": {"type": "string"}}
                },
                "required": ["tone_recommendation"]
            }
        },
        "required": ["situation_summary", "response_guidance"]
    }
}


def call_lmstudio_api(payload):
    headers = {
        "Content-Type": "application/json",
    }
    response = requests.post(lmstudio_url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed with status code {response.status_code}: {response.text}")


def _safe_jsonloads(maybe_str):
    if isinstance(maybe_str, (dict, list)):
        return maybe_str
    if not isinstance(maybe_str, str):
        return {"raw": maybe_str}
    try:
        return json.loads(maybe_str)
    except json.JSONDecodeError:
        return {"raw": maybe_str}


def promt_curator(prompt: str):
    payload = {
        "model": "curator",
        "messages": [{"role": "user", "content": str(prompt)}],
        "response_format": {"type": "json_schema", "json_schema": CURATOR_SCHEMA},
    }
    try:
        response = call_lmstudio_api(payload)
    except Exception as e:
        # Fallback: use plain text if server rejects schema
        if "response_format.type" in str(e):
            payload["response_format"] = {"type": "text"}
            response = call_lmstudio_api(payload)
        else:
            raise
    content = response.get("choices", [{}])[0].get("message", {}).get("content", {})
    return _safe_jsonloads(content)


def promt_thinker(instructions: str):
    thinker_payload = {
        "model": "thinker",
        "messages": [
            {
                "role": "system",
                "content": (
                    "Analyze the provided context and return JSON with keys: "
                    "internal_reasoning, verified_facts, assumptions_identified, patterns_observed, "
                    "information_gaps, user_emotional_state, situation_summary, response_guidance (tone_recommendation, "
                    "key_points_to_address, questions_to_explore, avoid_assumptions). Only return JSON."
                ),
            },
            {"role": "user", "content": str(instructions)},
        ],
        "response_format": {"type": "json_schema", "json_schema": THINKER_SCHEMA},
    }
    try:
        response = call_lmstudio_api(thinker_payload)
    except Exception as e:
        # Fallback: switch to text if schema not supported
        if "response_format.type" in str(e) or "Invalid JSON Schema" in str(e):
            thinker_payload["response_format"] = {"type": "text"}
            response = call_lmstudio_api(thinker_payload)
        else:
            raise
    content = response.get("choices", [{}])[0].get("message", {}).get("content", {})
    return _safe_jsonloads(content)


In [2]:
curator_response = promt_curator(test_prompt)
# print(curator_response)
for key, value in curator_response.items():
    print(f"{key}: {value}")

# We need to check if the "freshdata" is fresh

entities_mentioned: ['Buddy', 'stay command', 'dog trainer']
investigation_queries: [{'purpose': "Find past conversations about Buddy the dog and the 'stay' command to understand training progress timeline", 'collection': 'conversations', 'query': {'mongodb': {'user_input': {'$regex': 'Buddy'}, 'entities': {'$in': ['stay command']}, 'semantic_tags': {'$in': ['dog training', 'behavior command']}}}, 'timeframe': 'recent', 'priority': 4}, {'purpose': "Identify if there were previous mentions of the dog trainer or feedback from a trainer about Buddy's progress", 'collection': 'conversations', 'query': {'mongodb': {'user_input': {'$regex': 'dog trainer'}, 'entities': {'$in': ['stay command']}}}, 'timeframe': 'recent', 'priority': 3}, {'purpose': "Check if there are active projects or goals related to Buddy's training in user state", 'collection': 'user_state', 'query': {'mongodb': {'user_id': {'$regex': 'Buddy'}, 'active_projects': {'$regex': 'dog training'}, 'goals': {'$regex': 'stay comma

In [3]:
# Build a structured fake context to simulate retrieved data
fake_context_for_thinker = {
  "user_input": "The dog trainer said Buddy is making great progress with 'stay' command",
  "timestamp": "2025-08-20T14:30:00Z",
  "user_id": "user_123",
  "curator_analysis": {
    "entities_extracted": [
      {"name": "Buddy", "type": "person", "attributes": {"role": "dog", "status": "training"}},
      {"name": "stay command", "type": "concept", "attributes": {"category": "dog training", "progress": "positive"}}
    ],
    "primary_theme": "Dog training progress for Buddy",
    "emotional_context": "problem_solving",
    "user_intent": "User is sharing positive feedback about a training outcome",
    "conversation_trajectory": "follow_up"
  },
  "retrieved_context": {
    "previous_buddy_conversations": [
      {
        "timestamp": "2025-08-13T10:15:00Z",
        "user_input": "Had our first session with the professional trainer today. Buddy was pretty distracted but managed to hold 'sit' for about 3 seconds",
        "entities": ["Buddy", "trainer", "sit command"],
        "sentiment": "mixed",
        "tags": ["training", "milestone", "challenge"]
      },
      {
        "timestamp": "2025-08-06T16:45:00Z",
        "user_input": "Buddy keeps getting distracted during training. Maybe I need professional help?",
        "entities": ["Buddy", "training"],
        "sentiment": "frustrated",
        "tags": ["training", "problem", "seeking_help"]
      },
      {
        "timestamp": "2025-07-28T19:20:00Z",
        "user_input": "Started basic training with Buddy. He's smart but very energetic. 'Sit' is hit or miss.",
        "entities": ["Buddy", "training", "sit command"],
        "sentiment": "optimistic",
        "tags": ["training", "start", "basic_commands"]
      }
    ],
    "user_state_data": {
      "active_projects": {
        "buddy_training": {
          "start_date": "2025-07-28",
          "goal": "Basic obedience commands by September",
          "current_status": "working with professional trainer",
          "commands_target": ["sit", "stay", "come", "down"],
          "progress": {
            "sit": "improving",
            "stay": "in_progress",
            "come": "not_started",
            "down": "not_started"
          }
        }
      },
      "context_flags": ["first_time_dog_owner", "seeking_professional_help"]
    },
    "related_events": [
      {
        "timestamp": "2025-08-13T10:00:00Z",
        "type": "milestone",
        "description": "First professional training session",
        "entities_involved": ["Buddy", "trainer"],
        "tags": ["training", "professional", "first_session"],
        "outcome": "mixed_progress"
      },
      {
        "timestamp": "2025-08-06T16:00:00Z",
        "type": "decision",
        "description": "Decided to hire professional trainer",
        "entities_involved": ["Buddy"],
        "tags": ["training", "professional_help", "decision"],
        "sentiment": "determined"
      }
    ]
  },
  "information_gaps": [
    "How long has Buddy been working on 'stay' specifically?",
    "What does the trainer recommend for next steps?",
    "How does this progress compare to typical timelines?",
    "Is user satisfied with trainer choice and methodology?"
  ],
  "timeline_summary": {
    "july_28": "Started basic training, 'sit' inconsistent",
    "august_6": "Frustrated, decided to get professional help",
    "august_13": "First trainer session, some 'sit' progress",
    "august_20": "Trainer reports 'stay' progress (today)"
  }
}


In [4]:
# Convert the dict context into a readable string for the LLM
thinker_prompt = "Context:\n" + json.dumps(fake_context_for_thinker, ensure_ascii=False, indent=2)
print(thinker_prompt)

# Call the thinker model
thinker_response = promt_thinker(thinker_prompt)

# Print response in a readable way
if isinstance(thinker_response, dict):
    for key, value in thinker_response.items():
        print(f"{key}: {value}")
else:
    print(thinker_response)


Context:
{
  "user_input": "The dog trainer said Buddy is making great progress with 'stay' command",
  "timestamp": "2025-08-20T14:30:00Z",
  "user_id": "user_123",
  "curator_analysis": {
    "entities_extracted": [
      {
        "name": "Buddy",
        "type": "person",
        "attributes": {
          "role": "dog",
          "status": "training"
        }
      },
      {
        "name": "stay command",
        "type": "concept",
        "attributes": {
          "category": "dog training",
          "progress": "positive"
        }
      }
    ],
    "primary_theme": "Dog training progress for Buddy",
    "emotional_context": "problem_solving",
    "user_intent": "User is sharing positive feedback about a training outcome",
    "conversation_trajectory": "follow_up"
  },
  "retrieved_context": {
    "previous_buddy_conversations": [
      {
        "timestamp": "2025-08-13T10:15:00Z",
        "user_input": "Had our first session with the professional trainer today. Buddy was 

## Test Carlos pipeline end-to-end

This section uses the `Carlos` class to run: user input → curator → context assembly (Mongo) → thinker.


In [5]:
# Import and run Carlos pipeline

from carlos import Carlos
import json



# You can customize these via env vars if needed:

# import os; os.environ['LMSTUDIO_URL'] = 'http://localhost:1234/v1/chat/completions'

# os.environ['MONGODB_URI'] = 'mongodb://localhost:27017/carlos'



c = Carlos(user="notebook_user")

result = c.input("The dog trainer said Buddy is making great progress with 'stay' command")



# Show a compact summary

print("Sections:", list(result.keys()))

print("Curator keys:", list(result.get("curator", {}).keys())[:10])

print("Thinker fields:", [k for k in (result.get("thinker") or {}).keys()])

# Pretty print the Context, Curator, and Thinker outputs
try:
    print("\nContext JSON:\n", json.dumps(result.get("context"), ensure_ascii=False, indent=2))
except Exception:
    print(result.get("context"))

try:
    print("\nCurator JSON:\n", json.dumps(result.get("curator"), ensure_ascii=False, indent=2))
except Exception:
    print(result.get("curator"))

try:
    print("\nThinker JSON:\n", json.dumps(result.get("thinker"), ensure_ascii=False, indent=2))
except Exception:
    print(result.get("thinker"))


Sections: ['curator', 'context', 'thinker']
Curator keys: ['entities_mentioned', 'investigation_queries', 'context_focus']
Thinker fields: ['situation_summary', 'response_guidance', 'user_emotional_state']

Context JSON:
 {
  "user_input": "The dog trainer said Buddy is making great progress with 'stay' command",
  "user_id": "notebook_user",
  "timestamp": "2025-08-20T08:20:21.269928Z",
  "topic": null,
  "curator_analysis": {
    "context_focus": "Track the progression of Buddy's 'stay' command training, including prior efforts, user expectations, and milestone events, to provide a comprehensive view of training success and potential next steps.",
    "curiosity_analysis": null,
    "entities_extracted": []
  },
  "retrieved_context": {
    "recent_messages": [
      {
        "role": "user",
        "content": "The dog trainer said Buddy is making great progress with 'stay' command",
        "ts": "2025-08-20T08:20:16.214465Z"
      },
      {
        "role": "assistant",
        "c

## Index health and end-to-end tests
This section uses the `Carlos` class to inspect Mongo indexes and run full pipeline tests (curator → context → thinker → response).

In [6]:
# Index health summary
from carlos import Carlos
import json

c = Carlos(user="notebook_user")
health = c.index_health()
print("Collections:", ", ".join(sorted(health.keys())))

def _summarize(col, info):
    idx_names = [i.get('name') for i in info.get('indexes', [])]
    print(f"- {col}: count={info.get('count', 0)}, indexes={idx_names}")

for col, info in health.items():
    _summarize(col, info)

Collections: analyses, contexts, conversations, messages
- analyses: count=11, indexes=['_id_', 'user_ts', 'kind_ts', 'topic_ts']
- contexts: count=5, indexes=['_id_', 'user_ts', 'topic_ts']
- conversations: count=0, indexes=['_id_', 'user_input_text', 'entities', 'sentiment', 'user_ts', 'topic_ts']
- messages: count=6, indexes=['_id_', 'user_ts', 'ts_desc', 'topic_ts', 'content_text']


In [7]:
# End-to-end pipeline via Carlos.input()
test_msg = "The dog trainer said Buddy is making great progress with 'stay' command"
result = c.input(test_msg)

# Basic checks
assert isinstance(result, dict), "Pipeline did not return a dict"
for k in ("curator", "context", "thinker"):
    assert k in result, f"Missing section: {k}"

print("Sections:", list(result.keys()))
print("Curator keys:", list((result.get("curator") or {}).keys())[:10])
print("Thinker fields:", list((result.get("thinker") or {}).keys()))

# Pretty prints
try:
    print("\nContext JSON:\n", json.dumps(result.get("context"), ensure_ascii=False, indent=2))
except Exception:
    print(result.get("context"))
try:
    print("\nCurator JSON:\n", json.dumps(result.get("curator"), ensure_ascii=False, indent=2))
except Exception:
    print(result.get("curator"))
try:
    print("\nThinker JSON:\n", json.dumps(result.get("thinker"), ensure_ascii=False, indent=2))
except Exception:
    print(result.get("thinker"))

Sections: ['curator', 'context', 'thinker']
Curator keys: ['entities_mentioned', 'investigation_queries', 'context_focus']
Thinker fields: ['situation_summary', 'response_guidance', 'user_emotional_state']

Context JSON:
 {
  "user_input": "The dog trainer said Buddy is making great progress with 'stay' command",
  "user_id": "notebook_user",
  "timestamp": "2025-08-20T08:20:32.369017Z",
  "topic": null,
  "curator_analysis": {
    "context_focus": "Track the history of Buddy's training, especially progress on the 'stay' command, and identify user's emotional and project-based context around pet training.",
    "curiosity_analysis": null,
    "entities_extracted": []
  },
  "retrieved_context": {
    "recent_messages": [
      {
        "role": "user",
        "content": "The dog trainer said Buddy is making great progress with 'stay' command",
        "ts": "2025-08-20T08:20:27.219053Z"
      },
      {
        "role": "user",
        "content": "The dog trainer said Buddy is making g

In [8]:
# Chat() and debug info tests
chat_text = c.chat(test_msg)
print("Chat response:", chat_text)

dbg = c.get_debug_info("I'm thinking of getting a cat too")
print("Debug - Final response:", dbg.get("final_response"))
print("Debug - Tone used:", (dbg.get("response_generator") or {}).get("tone_used"))

Chat response: Oh, that's so wonderful to hear! 🐶 [laughs] Buddy’s making real progress with the 'stay'—that’s such a big win! I can just imagine him sitting there, eyes fixed, not moving a muscle... [sighs] It’s beautiful to see that kind of consistency. 💙

Have you noticed any specific milestones? Like, how long can he stay now? Or has he started holding the 'stay' in different spots—like the kitchen or outside? [clears throat]

Maybe we could set a little new challenge together? Like a 'stay' in a new room, or even with a distraction—like a toy or a treat being offered? What do you think? 🤔

What are your goals for Buddy’s training next? I’d love to hear what you’re dreaming for him!
Debug - Final response: Oh, that's such a wonderful idea! 🐾 I love how you're thinking about adding a cat to your life — it’s such a sweet, gentle step. Cats bring so much joy, quiet comfort, and even a little bit of silliness with every purr and meow.

They’re not just pets — they’re companions. You kn

### Optional: Bark TTS synthesis
This cell saves a WAV file from the latest chat response. Bark will download models on first run (can be large). Set `ENABLE_TTS = True` to enable.

In [9]:
# Preload bark models
import torch
import numpy as np

# Allowlist the offending global for safe loading
torch.serialization.add_safe_globals([np.core.multiarray.scalar])

# Patch torch.load to always use weights_only=False
import functools
_original_torch_load = torch.load
def _patched_torch_load(*args, **kwargs):
	kwargs.setdefault("weights_only", False)
	return _original_torch_load(*args, **kwargs)
torch.load = _patched_torch_load

from bark.generation import preload_models
preload_models()


C:\Users\sakki\AppData\Local\Temp\ipykernel_21680\4136678511.py:6: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  torch.serialization.add_safe_globals([np.core.multiarray.scalar])
No GPU being used. Careful, inference might be very slow!
No GPU being used. Careful, inference might be very slow!


coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

c:\Users\sakki\carlos\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to C:\Users\sakki/.cache\torch\hub\checkpoints\encodec_24khz-d7cc33bc.th


100%|██████████| 88.9M/88.9M [00:07<00:00, 12.5MB/s]



In [1]:
# Optional Bark TTS synthesis
ENABLE_TTS = True

if ENABLE_TTS:
    try:
        from IPython.display import Audio
        from bark import SAMPLE_RATE, generate_audio
        from scipy.io.wavfile import write as write_wav
        import os

        text = chat_text if 'chat_text' in globals() else dbg.get('final_response', 'Hello!')
        print("Synthesizing TTS for:", text[:120], "...")
        audio_array = generate_audio(text)
        out_path = os.path.abspath("carlos_response.wav")
        write_wav(out_path, SAMPLE_RATE, audio_array)
        print("Saved TTS to:", out_path)
    except Exception as e:
        print("TTS generation skipped:", e)
        print("Tip: pip install scipy  # if missing")

KeyboardInterrupt: 